In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [11]:
import bisect
N, L, R = map(int, input().split())
X = list(map(int, input().split()))
dp = [10 ** 30] * N
dp[0] = 0

size = 1
while size < N:
    size *= 2
node = [10 ** 30] * size * 2

def update(pos, x):
    pos += size
    node[pos] = x
    while pos >= 2:
        pos //= 2
        node[pos] = min(node[pos*2], node[pos*2+1])
        
def query(l, r, a, b, n):
    if r <= a or b <= l:
        return 10 ** 30
    
    if l <= a and b <= r:
        return node[n]
    
    mid = (a + b) // 2
    vl = query(l, r, a, mid, n*2)
    vr = query(l, r, mid, b, n*2+1)
    return min(vl, vr)

update(0, 0)
for i in range(1, N):
    now = X[i]
    l = now - R
    r = now - L
    l_idx = bisect.bisect_left(X, l)
    r_idx = bisect.bisect_right(X, r)
    dp[i] = query(l_idx, r_idx, 0, size, 1) + 1
    update(i, dp[i])
print(dp[N-1])

2


In [12]:
dp

[0, 1, 1, 2, 2]